In [ ]:
# written by Maximilian Ullherr, maximilian.ullherr@physik.uni-wuerzburg.de,
# Lehrstuhl fuer Roentgenmikroskopie/Universitaet Wuerzburg, Josef-Martin-Weg 63, 97074 Wuerzburg, Germany
# last updated 2019-08-08

%pylab
import sys; sys.path.append('..')  # copy the pyETX module either into the folder with the notebook and remove this line
from SNR_u_evaluation import *

print('\n=== printing the docstring for the main function ===')
help(estimate_SNR)

## Contents

1\. Simulation - shows how this works and show example evaluation

    a) generate test data
    
    b) estimate SNR without flats/refs+darks
    
    c) estimate SNR with flats/refs but no darks
    
    d) inaccurate SNR estimate without pixel defect removal
    
    e) configure pixel defect removal
    
    f) generate test data with darks
    
    g) estimate SNR with flats/refs and darks
   
2\. optimal measurement parameters

*some simulations to choose the correct measurement parameters*
   
# 1. Simulation

## simple, no refs/flats

Warning: Do not use this for images that are already flat/dark corrected, it would result in reduced accuracy.

In [ ]:
# a) generate test data
base_intensity = 100
test_projs, test_proj = generate_test_data(base_intensity=base_intensity, add_pixel_defects=False, shape=(700, 700))
    
test_flats = random.poisson(ones_like(test_projs)*base_intensity)
show_image(test_projs.mean(axis=0), figsize=(5, 5))

In [ ]:
# b) estimate SNR without flats/refs+darks

u, S, N, SNR, avg_image = estimate_SNR(test_projs, save_path='test_SNR_result')


PS = PowerSpectrumCalculator(test_projs.shape[1:])
S_coord, S_actual = PS.radial_power_spectrum(test_proj)

fig, axes = plt.subplots(1, 3, figsize=(12, 4), tight_layout=True)

axes[0].semilogy(u, SNR, label='computed SNR')
axes[0].semilogy(u, S_actual/base_intensity, label='actual')
axes[0].set_ylim(detect_lower_SNR_limit(SNR), None)
axes[1].semilogy(u, S, label='computed signal power spectrum')
axes[1].semilogy(u, S_actual, label='actual')
axes[1].set_ylim(detect_lower_SNR_limit(S), None)
axes[2].plot(u, N, label='computed noise power spectrum')
axes[2].plot(u, ones_like(N)*mean(N), label='mean of computed')
axes[2].plot(u, ones_like(N)*sqrt(mean(test_proj**2)), label='actual')

[ax.legend() for ax in axes]; [ax.grid() for ax in axes]
[ax.set_xlabel('spatial frequency') for ax in axes]
[axes[k].set_title(t) for k, t in enumerate(('SNR', 'signal power spectra', 'noise'))]

display(fig)

## with refs/flats

In [ ]:
# c) estimate SNR with flats/refs but no darks

u, S, N, SNR, mean_im = %time estimate_SNR(test_projs, refs=test_flats)

PS = PowerSpectrumCalculator(test_projs.shape[1:])
u, S_actual = PS.radial_power_spectrum(test_proj/base_intensity)

fig, axes = plt.subplots(1, 3, figsize=(12, 4), tight_layout=True)

axes[0].semilogy(u, SNR, label='computed SNR')
axes[0].semilogy(u, S_actual*base_intensity, label='actual')
axes[0].set_ylim(detect_lower_SNR_limit(SNR), None)
axes[1].semilogy(u, S, label='computed signal power spectrum')
axes[1].semilogy(u, S_actual, label='actual')
axes[1].set_ylim(detect_lower_SNR_limit(S), None)
axes[2].plot(u, N, label='computed noise power spectrum')
axes[2].plot(u, ones_like(N)*mean(N), label='mean of computed')
axes[2].plot(u, ones_like(N)/sqrt(mean(test_proj**2)), label='actual')

[ax.legend() for ax in axes]; [ax.grid() for ax in axes]
[ax.set_xlabel('spatial frequency') for ax in axes]
[axes[k].set_title(t) for k, t in enumerate(('SNR', 'signal power spectra', 'noise'))]

display(fig)

## signal artifacts example: no pixel defect removal

#### ==== WARNING ==== 

The SNR estimate will be highly inaccurate if pixel defects are present but not removed. 

This will typically have the effect that the SNR appears to be constantly high over most of the frequency range.

In actual applications, this effect is usually less severe than in this example.

These artifacts can often not be fully suppressed. If they are reduced to a magnitude of 0.001, it is often sufficiently good.

In [ ]:
# d) inaccurate SNR estimate without pixel defect removal
test_projs_f = copy(test_projs)
fmax(test_projs_f, 1, out=test_projs_f)  # remove NaN

u, S, N, SNR, mean_im = %time estimate_SNR(test_projs_f, refs=test_flats)

PS = PowerSpectrumCalculator(test_projs.shape[1:])
u, S_actual = PS.radial_power_spectrum(test_proj/base_intensity)

fig, axes = plt.subplots(1, 3, figsize=(12, 4), tight_layout=True)

axes[0].semilogy(u, SNR, label='computed SNR')
axes[0].semilogy(u, S_actual*base_intensity, label='actual')
axes[0].set_ylim(detect_lower_SNR_limit(SNR), None)
axes[1].semilogy(u, S, label='computed signal power spectrum')
axes[1].semilogy(u, S_actual, label='actual')
axes[1].set_ylim(detect_lower_SNR_limit(S), None)
axes[2].plot(u, N, label='computed noise power spectrum')
axes[2].plot(u, ones_like(N)*mean(N), label='mean of computed')
axes[2].plot(u, ones_like(N)/mean(test_proj), label='actual')

[ax.legend() for ax in axes]; [ax.grid() for ax in axes]
[ax.set_xlabel('spatial frequency') for ax in axes]
[axes[k].set_title(t) for k, t in enumerate(('SNR', 'signal power spectra', 'noise'))]

display(fig)
print('this is NOT how an evaluation should be done and the results are inaccurate')
print('note that noise is still accurate, only the signal is distorted')

## pixel defect removal configuration

pixel defects should usually account for less than 1 % of the pixels, here we have 0.67 %.

In [ ]:
# e) configure pixel defect removal
filterer = ImageSeriesPixelArtifactFilterer()
#filterer.defect_threshold = np.sqrt(base_intensity/test_projs.shape[0])*10
filterer.verbose = True

filterer.defect_threshold = 0.001  # too low, produces warning
filterer.test(test_projs, test_flats)

filterer.defect_threshold = 0.3  # good
filterer.test(test_projs, test_flats)

filterer.defect_threshold = 100  # too high, no warning (!)
filterer.test(test_projs, test_flats)
print('no warning, but ineffective filter')

filterer.defect_threshold = 0.3
test_projs_f = %time filterer(test_projs/test_flats.mean(axis=0))

display(show_image(test_projs.mean(axis=0), figsize=(5, 5)))
display(show_image(test_projs_f.mean(axis=0), figsize=(5, 5)))

## with refs/flats and darks

In [ ]:
# f) generate test data with darks
base_intensity = 300
dark_intensity = 5

test_projs, test_proj = generate_test_data(base_intensity=base_intensity)

test_flats = random.poisson(ones_like(test_projs)*(base_intensity+dark_intensity))
test_darks = random.poisson(ones_like(test_projs)*dark_intensity)
test_projs += random.poisson(ones_like(test_projs)*dark_intensity)

show_image(test_projs[0]/test_flats.mean(axis=0), figsize=(5, 5))

In [ ]:
# g) estimate SNR with flats/refs and darks
filterer = ImageSeriesPixelArtifactFilterer()
filterer.verbose = True

u, S, N, SNR, mean_im = %time estimate_SNR(test_projs, refs=test_flats, darks=test_darks, series_filterer=filterer)

PS = PowerSpectrumCalculator(test_projs.shape[1:])
u, S_actual = PS.radial_power_spectrum(test_proj/base_intensity)

fig, axes = plt.subplots(1, 3, figsize=(12, 4), tight_layout=True)

axes[0].semilogy(u, SNR, label='computed SNR')
axes[0].semilogy(u, S_actual*base_intensity, label='actual')
axes[0].set_ylim(detect_lower_SNR_limit(SNR), None)
axes[1].semilogy(u, S, label='computed signal power spectrum')
axes[1].semilogy(u, S_actual, label='actual')
axes[1].set_ylim(detect_lower_SNR_limit(S), None)
axes[2].plot(u, N, label='computed noise power spectrum')
axes[2].plot(u, ones_like(N)*mean(N), label='mean of computed')
axes[2].plot(u, ones_like(N)/mean(test_proj), label='actual (with dark noise)')

[ax.legend() for ax in axes]
[ax.set_xlabel('spatial frequency') for ax in axes]
[axes[k].set_title(t) for k, t in enumerate(('SNR', 'signal power spectra', 'noise'))]

display(fig)

# 2. optimal measurement parameters

the aim is to reduce the errors while keeping measurement times low

note that different signal strengths change the optimal setup

### conclusion

- a higher number of projections (=images) is translated almost linearly to a better lower limit (increasing both number of flats and number of images)
- a higher number of counts only has a small effect on the lower limit
- a very low intensity (e.g. 30 average counts) in the individual images is sufficient if a high number of images is used
- taking a higher number of images is more time efficient than longer exposure times
- taking more flats/refs than images is good for precision

### recommendations

- 100 images per SNR measurement are recommended for a high precision. If this is impossible, take as many images as reasonably possible.
- 100 counts per image are probably sufficient, but do not go above 500 counts.
- Less than 1000 counts in sum (e.g. 25x40) is not recommended.
- The product of the number of images and the counts should be larger than the number of counts in a normal image
- If the flats/refs are reused for several image series, then take many more flats/refs than images. Example: 10 parameters with 100 images each and 300 flats/refs.

### suggested setups (examples)

- high precision: 100 images with 100-200 counts each
- medium precision: 60 images with 60-200 counts each
- low precisions: 30 images with 40-200 counts each

In [ ]:
def geometric_mean_diff(arr1, arr2):
    return exp(np.nanmean(np.abs(log(arr1)-log(arr2))))

def estimate_SNR_error(SNR_estimate, SNR_actual):
    lower_limit = detect_lower_SNR_limit(SNR_estimate)
    loc = SNR_estimate > lower_limit*3
    loc &= SNR_estimate < lower_limit*1e2
    gavg_err = geometric_mean_diff(SNR_estimate[loc], SNR_actual[loc])
    return gavg_err, lower_limit

def estimate_SNR_error_comparable(SNR_estimate, SNR_actual, max_bin):
    lower_limit = detect_lower_SNR_limit(SNR_estimate)
    gavg_err = geometric_mean_diff(SNR_estimate[:max_bin], SNR_actual[:max_bin])
    return gavg_err, lower_limit

base_intensities = (10, 30, 100, 300, 1000)
nprojs = (2, 10, 50, 250)
#base_intensities = (10, 30)
#nprojs = (10, 100)
max_nprojs = max(nprojs)
print('WARNING: low_limit and err have a very low precision and vary a lot from run to run')

for base_intensity in base_intensities:
    test_projs, test_proj = generate_test_data((500, 500), base_intensity=base_intensity, nproj=max_nprojs, 
                                               add_pixel_defects=False)
    test_flats = random.poisson(ones_like(test_projs)*base_intensity)
    PS = PowerSpectrumCalculator(test_projs.shape[1:])
    S_coord, S_actual = PS.radial_power_spectrum(test_proj/base_intensity)
    print('generated data')
    if base_intensity < 50: # fix for division by zero
        fmax(1, test_projs, out=test_projs)
        fmax(1, test_flats, out=test_flats)
    for nproj in nprojs:
        low_limit = 0.
        gavg_err = 0.
        u_bins, S, N, SNR, mean_im = estimate_SNR(test_projs[:nproj], refs=test_flats[:nproj])
        gavg_err, low_limit = estimate_SNR_error_comparable(SNR, S_actual*base_intensity, 12)
        gavg_err -= 1
        if low_limit is None: low_limit = 0.0
        
        print(f'{base_intensity:4} counts x {nproj:4} images: low_limit = {low_limit:7.4e}; err = {gavg_err*100:7.1f} %')

In [ ]:
nprojs = (2, 3, 10, 30, 100, 300, 1000)
max_nprojs = max(nprojs)
nproj_other = 100
print('WARNING: low_limit and err have a very low precision and vary a lot from run to run')

base_intensity = 100
test_projs, test_proj = generate_test_data((300, 300), base_intensity=base_intensity, nproj=max_nprojs, 
                                           add_pixel_defects=False)
test_flats = random.poisson(ones_like(test_projs)*base_intensity)
PS = PowerSpectrumCalculator(test_projs.shape[1:])
S_coord, S_actual = PS.radial_power_spectrum(test_proj/base_intensity)
print('generated data')
if base_intensity < 50: # fix for division by zero
    fmax(1, test_projs, out=test_projs)
    fmax(1, test_flats, out=test_flats)
    
for nproj in nprojs:
    u_bins, S, N, SNR, mean_im = estimate_SNR(test_projs[:nproj], 
                                              refs=test_flats[:nproj_other])

    gavg_err, low_limit = estimate_SNR_error_comparable(SNR, S_actual*base_intensity, 12)
    gavg_err -= 1
    print(f'{base_intensity:4} counts x [{nproj:4} images, {nproj_other:4} flats]: low_limit = {low_limit:7.4e}; err = {gavg_err*100:7.1f} %')    

for nproj in nprojs:
    u_bins, S, N, SNR, mean_im = estimate_SNR(test_projs[:nproj_other], 
                                              refs=test_flats[:nproj])

    gavg_err, low_limit = estimate_SNR_error_comparable(SNR, S_actual*base_intensity, 12)
    gavg_err -= 1

    print(f'{base_intensity:4} counts x [{nproj_other:4} images, {nproj:4} flats]: low_limit = {low_limit:7.4e}; err = {gavg_err*100:7.1f} %')

In [ ]:
base_intensity = 100
nprojs         = (2, 10, 50, 250)
max_nprojs = max(nprojs)

fig, ax = plt.subplots(1, 1, figsize=(6, 6), tight_layout=True)

test_projs, test_proj = generate_test_data((300, 300), base_intensity=base_intensity, nproj=max_nprojs, 
                                           add_pixel_defects=False)
test_flats = random.poisson(ones_like(test_projs)*base_intensity)
PS = PowerSpectrumCalculator(test_projs.shape[1:])
u, S_actual = PS.radial_power_spectrum(test_proj/base_intensity)
print('generated data')
ax.semilogy(u, S_actual*base_intensity, label='actual')

for nproj in nprojs:
    u, S, N, SNR, mean_im = estimate_SNR(test_projs[:nproj], refs=test_flats[:nproj])
    
    fmax(SNR, detect_lower_SNR_limit(SNR), out=SNR)
    ax.semilogy(u, SNR, label=f'computed SNR, nproj = {nproj}')

ax.legend()
ax.set_xlabel('spatial frequency')
ax.set_ylabel('SNR (clipped to lower limit)')

plt.show(fig)

In [ ]:
base_intensities = (10, 30, 100, 300)
nproj      = 50
max_nprojs = max(nprojs)
plotted_actual = False

fig, ax = plt.subplots(1, 1, figsize=(6, 6), tight_layout=True)


for base_intensity in base_intensities:
    test_projs, test_proj = generate_test_data((300, 300), base_intensity=base_intensity, nproj=max_nprojs, 
                                               add_pixel_defects=False)
    test_flats = random.poisson(ones_like(test_projs)*base_intensity)
    if base_intensity < 50: # fix for division by zero
        fmax(1, test_projs, out=test_projs)
        fmax(1, test_flats, out=test_flats)
    print('generated data')
    
    if not plotted_actual:
        PS = PowerSpectrumCalculator(test_projs.shape[1:])
        u, S_actual = PS.radial_power_spectrum(test_proj/base_intensity)
        ax.semilogy(u, S_actual*100, label='actual')
        plotted_actual = True
        
    u, S, N, SNR, mean_im = estimate_SNR(test_projs, refs=test_flats)
    fmax(SNR, detect_lower_SNR_limit(SNR), out=SNR)
    ax.semilogy(u, SNR/base_intensity*100, label=f'computed SNR (scaled to 100 counts), {base_intensity} counts')

ax.legend()
ax.set_xlabel('spatial frequency')
ax.set_ylabel('SNR (clipped to lower limit)')

plt.show(fig)

In [ ]:
plt.close('all')